In [1]:
# Import the required libraries.
import os
import cv2
#import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

#from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
tf.keras.backend.clear_session()
seed_constant = 27
np.random.seed(seed_constant)
tf.random.set_seed(seed_constant)
random.seed(seed_constant)
tf.keras.utils.set_random_seed(seed_constant)

In [3]:
# Create a Matplotlib figure and specify the size of the figure.
plt.figure(figsize = (20, 20))

# Get the names of all classes/categories in UCF50.
TRAIN_DIR = r'train'
VAL_DIR = r'val'
TEST_DIR = r'test'
all_classes_names = sorted(os.listdir(TRAIN_DIR))
print(all_classes_names)


['1outfinal', '2outfinal', '3outfinal', '4outfinal']


<Figure size 2000x2000 with 0 Axes>

In [4]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 80, 80

# Specify the directory containing the UCF50 dataset. 
#DATASET_DIR = "/newdatasetMediapipe/kata/"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST =all_classes_names
#CLASSES_LIST = ['2 Tolong', '7 permisi', '1 Maaf']

In [5]:
SEQUENCE_LENGTH = 40#getAverageSequenceLength()
print(SEQUENCE_LENGTH)

40


In [6]:
def getDatafromNpz(file):
    return file['data'],file['label']
dtrain = np.load('feature_train.npz')
dval = np.load('feature_val.npz')
dtest = np.load('feature_test.npz')

features_train,labels_train = getDatafromNpz(dtrain)
features_test,labels_test = getDatafromNpz(dtest)
features_val,labels_val = getDatafromNpz(dval)

In [7]:
def create_convlstm_model():
    '''
    This function will construct the required convlstm model.
    Returns:
        model: It is the required constructed convlstm model.
    '''

    # We will use a Sequential model for model construction
    model = Sequential()

    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last", dropout=0.3,
                          return_sequences=True, input_shape = (SEQUENCE_LENGTH,
                                                                                      IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same', data_format='channels_last'))
    #model.add(TimeDistributed(Dropout(0.3)))
    
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last", dropout=0.3,
                          return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same', data_format='channels_last'))
    #model.add(TimeDistributed(Dropout(0.3)))
    
    #model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
    #                     dropout=0.2, return_sequences=True))
    
    #model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    #model.add(TimeDistributed(Dropout(0.2)))
    
    
    model.add(Flatten()) 
    
    model.add(Dense(len(CLASSES_LIST), activation = "softmax"))
    
    ########################################################################################################################
     
    # Display the models summary.
    model.summary()
    
    # Return the constructed convlstm model.
    return model

In [ ]:
def create_convlstm_model():
    #tf.keras.backend.clear_session()
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''
    IMAGE_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH) + (3,)
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet',  include_top=False)
    #base_model.trainable = False
    # the 4 last layers (my own choice, up to you)
    for layer in base_model.layers:
        layer.trainable = False
    
    inputs = Input(shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    x = TimeDistributed(base_model)(inputs)
    #x = TimeDistributed(Dense(192))(x)

    #the ouput of the model is [1, 1, 2048], need to squeeze
    #x = Lambda(lambda x: tf.squeeze(tf.squeeze(x,2),2))(x)
    x = TimeDistributed(Flatten())(x)
    x = Bidirectional(LSTM(16, return_sequences=False))(x)
    #x = BatchNormalization()(x)
    """x = LSTM(100, activation='relu', return_sequences=True)(x)
    x = Dense(100, activation = 'relu')(x)
    x = Dropout(0.25)(x)
    x = LSTM(100, activation='relu', return_sequences=False)(x)
    x = Dense(50, activation = 'relu')(x)"""
    
    outputs = Dense(len(CLASSES_LIST), activation = 'softmax')(x)
    model = tf.keras.Model(inputs, outputs)

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [ ]:
#The best
def create_convlstm_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################

    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(TimeDistributed(MaxPooling2D((2, 2)))) 
    model.add(Dropout(0.3))
    
    model.add(TimeDistributed(Conv2D(
        32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(Dropout(0.3))
    
    #model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    #model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(Dropout(0.3))
    
    #model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    #model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))#GOAP
    #model.add(Dropout(0.4))                               
    model.add(LSTM(64, return_sequences=False))#32
    #model.add(BatchNormalization())
    model.add(Dropout(0.3))
                                      
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [8]:
# Construct the required convlstm model.
convlstm_model = create_convlstm_model()

# Display the success message. 
print("Model Created Successfully!")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 40, 78, 78, 8)     3200      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 20, 39, 39, 8)    0         
 )                                                               
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 20, 37, 37, 16)    13888     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 10, 19, 19, 16)   0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 57760)             0         
                                                                 
 dense (Dense)               (None, 4)                 2

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback

from timeit import default_timer as timer

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

model_ckpt = ModelCheckpoint('MyNet.h5',monitor='val_accuracy', verbose=1,save_best_only=True )
# Compile the model and specify loss function, optimizer and metrics values to the model
adam=tf.keras.optimizers.Adam(learning_rate=0.0005)
convlstm_model.compile(loss = 'categorical_crossentropy',optimizer=adam, metrics = ["accuracy"])
#tf.keras.optimizers.Adam(lr=0.0005)
# Start training the model.
convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs = 100, batch_size = 4,
                                                     validation_data=(features_val, labels_val),
                                                     callbacks = [cb,model_ckpt])
                                                     #callbacks = [TerminateOnBaseline(monitor='val_accuracy', baseline=0.96)])

In [ ]:
print(cb.logs)
print(sum(cb.logs))
np.save('my_history.npy',convlstm_model_training_history.history)

In [ ]:
history=np.load('my_history.npy',allow_pickle='TRUE').item()

In [ ]:
convlstm_model_training_history = history

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
    '''
    This function will plot the metrics passed to it in a graph.
    Args:
        model_training_history: A history object containing a record of training and validation 
                                loss values and metrics values at successive epochs
        metric_name_1:          The name of the first metric that needs to be plotted in the graph.
        metric_name_2:          The name of the second metric that needs to be plotted in the graph.
        plot_name:              The title of the graph.
    '''
    
    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history[metric_name_1]
    metric_value_2 = model_training_history[metric_name_2]
    
    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))

    # Add legend to the plot.
    plt.legend()
    
    plt.savefig("output"+metric_name_1+".jpg")

In [ ]:
# Visualize the training and validation loss metrices.
plot_metric(convlstm_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
# Visualize the training and validation accuracy metrices.
plot_metric(convlstm_model_training_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy') 

In [9]:
# Evaluate the trained model.
from keras.models import load_model
convlstm_model = load_model("MyNet.h5")
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)


5/5 [==============================] - 4s 137ms/step - loss: 0.3954 - accuracy: 0.9062


In [10]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'convlstm_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

# Save your Model.
convlstm_model.save(model_file_name)

In [11]:
predict_x=convlstm_model.predict(features_test) 
rounded_predictions=np.argmax(predict_x,axis=1)

print(rounded_predictions[1])

rounded_labels=np.argmax(labels_test, axis=1)
print(rounded_labels[1])
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(rounded_labels, rounded_predictions)
print(cm)


5/5 [==============================] - 1s 135ms/step
1
3
[[40  0  0  0]
 [ 0 32  4  4]
 [ 1  3 36  0]
 [ 0  2  1 37]]


In [12]:
from sklearn.metrics import classification_report
# predict 
predictions = convlstm_model.predict(features_test)#, batch_size = 4)
pred = np.argmax(predictions, axis=1)
# label
rounded_labels=np.argmax(labels_test, axis=1)

print(classification_report(rounded_labels, pred))
# precission “Berapa persen mahasiswa yang benar DO dari keseluruhan mahasiswa yang diprediksi DO?”
# recall "Berapa persen mahasiswa yang diprediksi DO dibandingkan keseluruhan mahasiswa yang sebenarnya DO?"

5/5 [==============================] - 1s 137ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        40
           1       0.86      0.80      0.83        40
           2       0.88      0.90      0.89        40
           3       0.90      0.93      0.91        40

    accuracy                           0.91       160
   macro avg       0.91      0.91      0.91       160
weighted avg       0.91      0.91      0.91       160

